In [66]:
import json
import base64
from functools import partial
from pathlib import Path
import numpy as np
import imageio
import pandas as pd
from bqplot import (
    Figure,
    Scatter,
    LinearScale,
    Axis,
    Toolbar,
    Label,
    ColorAxis,
    ColorScale,
    OrdinalColorScale
)
import ipywidgets as widgets
from IPython.display import display


In [77]:
# Thanks https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916

def html_thumb(thumb):
    return f"""<img src="data:image/jpg;base64,{thumb}">"""

class MeasureViz(widgets.VBox):
    
    def __init__(self, measure_hdf, thumbs_json):
        
        measures = pd.read_hdf(measure_hdf)
        channels = measures["C"]
        self.measures = measures.set_index(['Id', 'C', "Z", "T"])
        self.sheet = widgets.Output()
        
        with open(thumbs_json, 'r') as fh:
            thumbs = json.load(fh)
        self.thumbs = {int(k): th for k, th in thumbs.items()}
        self.goto = widgets.HTML('')
        x_sc = LinearScale()
        y_sc = LinearScale()
        c_sc = ColorScale(scheme="PuBuGn")
    
        c_ord = OrdinalColorScale(colors=['DodgerBlue', 'SeaGreen', 'HotPink', 'OrangeRed'])
        self.scat = Scatter(
            x=self.measures["SNR"],
            y=self.measures["resolution"].clip(0, 20),
            color=channels,
            scales={
                "x": x_sc,
                "y": y_sc,
                "color": c_ord,
            },
            names=self.measures.index.to_list(),
            display_names=False,
            fill=True,
            default_opacities=[0.6,],
        )
        ax_x = Axis(
            scale=x_sc,
            label="SNR",
            tick_format='0.2f'
        )
        ax_y = Axis(
            scale=y_sc,
            label='Resolution',
            orientation="vertical"
        )
    
        self.scat.on_element_click(self.goto_db)
        self.scat.on_element_click(self.show_data)
        self.scat.on_hover(self.show_thumb)
        self.scat.tooltip = widgets.HTML("")

        self.fig = Figure(
            marks=[self.scat,],
            axes=[ax_x, ax_y],
        )
        super().__init__([self.fig, widgets.HBox([self.goto, self.sheet])])
        
    def show_thumb(self, cbk, target):
        name = target['data']["name"]
        self.scat.tooltip = widgets.HTML(html_thumb(self.thumbs.get(name["0"], "")))

    def goto_db(self, cbk, target):
        name = target['data']["name"]
        idx = name["0"]
        html_ = html_thumb(self.thumbs.get(idx, ""))
        if html_:
            url = f"http://139.124.81.38:4080/webclient/img_detail/{idx}"
            self.goto.value = (f'<p><hr></p><a href={url} target="_blank">{html_}</a><h4><hr>n°{idx}<hr></h4>')

    def show_data(self, cbk, target):
        self.sheet.clear_output()
        name = target['data']["name"]
        mv.measures.loc[16437]
        _highlight = partial(highlight_, idx=tuple(name.values())[1:])
        with self.sheet:
            display(self.measures.loc[name["0"]].style.apply(_highlight))
            
def highlight_(df, idx):
    return [
        'background: lightblue' if idx_ == idx
        else '' for idx_, _ in df.iteritems()
    ]
            


In [78]:
base = Path("/home/guillaume/AmuBox/DataExchange/")
measure_hdf = base / "measures_10452_2020-04-20.hf5"
thumbs = base / "thumbs.json"

mv = MeasureViz(measure_hdf, thumbs)
mv

MeasureViz(children=(Figure(axes=[Axis(label='SNR', scale=LinearScale(), tick_format='0.2f'), Axis(label='Reso…